# lấy link url sale

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import csv
import openpyxl

print('- Finish importing packages')

# Task 1.1: Open Chrome and Access Batdongsan.vn
driver = webdriver.Chrome()
sleep(2)
url = 'https://batdongsan.vn/moi-gioi/nha/p1'
driver.get(url)
print('- Finish initializing a driver')
sleep(2)

# Load all links of sales
def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('div', class_= 'more')
    all_profile_URL = []
    for profile in profiles:
        profile_URL = profile.find('a').get('href')
        if profile_URL not in all_profile_URL:
            all_profile_URL.append(profile_URL)
    return all_profile_URL

# Define the number of pages to scrape
number_of_page = 288

# Create an empty list to store all profile URLs
all_profile_URL = []

# Loop over each page and extract the URLs
for page in range(1, number_of_page+1):
    # Get the URL of the current page
    url = f'https://batdongsan.vn/moi-gioi/nha/p{page}'
    
    # Load the page
    driver.get(url)
    sleep(2)
    
    # Extract the URLs on the current page
    URLs_one_page = GetURL()
    all_profile_URL.extend(URLs_one_page)

# Create a new Excel workbook
wb = openpyxl.Workbook()

# Create a new worksheet
ws = wb.create_sheet("url_sale")

# Write the list to the worksheet
for row in all_profile_URL:
    ws.append([row])

# Save the workbook
wb.save("link_sale.xlsx")
print('done ')

# truy cập link lấy thông tin sale

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import csv
import openpyxl 
    
#Khởi tạo trình duyệt Chrome
driver = webdriver.Chrome()

# Mở file Excel chứa các URL
wb = openpyxl.load_workbook('link_url.xlsx')

# Chọn sheet đầu tiên
ws = wb.active

# Tạo một danh sách để lưu email và số điện thoại
result_list = []

# Lặp qua từng hàng trong file Excel, bắt đầu từ dòng thứ 2 (do dòng đầu tiên là tiêu đề)
for row in ws.iter_rows(min_row=2):
    url = row[0].value
    
    # Truy cập vào trang bất động sản
    driver.get(url)
    sleep(2)

    # Lấy thông tin liên lạc từ trang
    page_source = BeautifulSoup(driver.page_source)
    infomation = page_source.find_all('div', class_='meta')
    a_tags = infomation[0].find_all('a')
    if len(a_tags) >= 2:
        email = a_tags[0].get('href').split(':')[1]
        phone_number = a_tags[1].text.replace('[^0-9]', '')
    else:
        email = ''
        phone_number = ''
    name = page_source.find('h1', class_='name').text.strip()

    # Thêm email và số điện thoại vào danh sách kết quả
    result_list.append([name, email, phone_number])

# Tạo một file Excel mới để lưu danh sách kết quả
result_wb = openpyxl.Workbook()
result_ws = result_wb.active

# Ghi danh sách kết quả vào file Excel mới
for row in result_list:
    result_ws.append(row)

# Lưu file Excel mới
result_wb.save('sale_batdongsan_com.xlsx')

# Đóng trình duyệt
driver.quit()
